## Imports

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('__file__'), '..', 'src')))

#%pip install -r requirements.txt
from data_preprocessing import data_preprocessing_tumor
from model import BrainCNN


import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchbearer
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchbearer import Trial
from torchmetrics import Precision, Recall, F1Score
from torchbearer.callbacks import EarlyStopping

Stochastic Gradient Descent (SGD) optimiser  
categorical cross entropy as the loss function
learning rate of 0.001 and
categorical accuracy as the evaluation metric
Model Accuracy and 
Model Loss respectively. 
This model was trained for 100 epochs,
but if the validation loss fails to improve, 
the training process is terminated using an approach known as early stopping.

In [ ]:
trainLoader, testLoader = data_preprocessing_tumor()
model = BrainCNN()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimizer, loss_function, metrics=['loss', 'cat_accuracy', 'accuracy']).to(device)
early_stopping = EarlyStopping(monitor='loss', patience=10, mode='min', verbose=1)
trial.with_generators(trainLoader, test_generator=testLoader)

trial.run(100, callbacks=[early_stopping])
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)

torch.save(model.state_dict(), "./braincnn_prototype.weights")


In [ ]:
precision = Precision(num_classes=2, average='macro')  # Замените 2 на количество классов
recall = Recall(num_classes=2, average='macro')  # Замените 2 на количество классов
f1_score = F1Score(num_classes=2, average='macro')  # Замените 2 на количество классов

# Извлечение предсказаний и меток для оценки дополнительных метрик
model.eval()  # Устанавливаем модель в режим оценки
y_true = []
y_pred = []

with torch.no_grad():  # Отключаем вычисление градиентов
    for inputs, targets in testLoader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)  # Получаем индексы максимальных вероятностей (классов)
        
        y_true.extend(targets.cpu().numpy())  # Сохраняем истинные метки
        y_pred.extend(predicted.cpu().numpy())  # Сохраняем предсказанные метки

# Переводим в тензоры
y_true = torch.tensor(y_true)
y_pred = torch.tensor(y_pred)

# Обновляем метрики
precision.update(y_pred, y_true)
recall.update(y_pred, y_true)
f1_score.update(y_pred, y_true)

# Получаем значения метрик
print("Precision:", precision.compute())
print("Recall:", recall.compute())
print("F1 Score:", f1_score.compute())
